In [ ]:
!pip install transformers
!pip install transformers[torch]

!pip install transformers[torch] accelerate -U datasets torch

In [ ]:
!pip install torch transformers

In [ ]:
import os
import json
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from google.colab import drive

In [ ]:
# 구글 드라이브 마운트
drive.mount('/content/drive')

# 데이터 경로 설정 (구글 드라이브 내의 경로)
data_path = '/content/drive/MyDrive/01.데이터/1.Training/라벨링데이터'



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 모든 JSON 파일의 경로를 리스트로 저장
json_files = [os.path.join(data_path, f) for f in os.listdir(data_path) if f.endswith('.json')]

# 데이터를 저장할 리스트
all_texts = []

In [ ]:
len(json_files)

51690

In [ ]:
# 각 JSON 파일을 불러와서 데이터 추가                           #json 파일에서 'info'탭의 'annotations'와 'text'컬럼에서 대화만 뽑아 all_texts에 저장
for file_path in json_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        for item in data['info']:
            text = item['annotations']['text']
            all_texts.append(text)

print(f'Loaded {len(all_texts)} dialogues.')

KeyboardInterrupt: 

In [ ]:
len(all_texts)

4942

In [ ]:



# 텍스트 데이터를 결합하여 GPT 모델 입력용 텍스트 생성
dataset_text = "\n".join(all_texts)

# 토크나이저 설정
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')  #는 Hugging Face의 Transformers 라이브러리에서 제공하는 코드로, GPT-2 모델에 맞는 토크나이저를 로드,from_pretrained 메서드 사용: 이 메서드는 지정된 이름('gpt2')에 해당하는 사전 훈련된 토크나이저를 다운로드하고 로드합니다.
tokenizer.pad_token = tokenizer.eos_token



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded 0 dialogues.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
all_texts

[]

In [ ]:
# 토큰화된 데이터 생성
inputs = tokenizer(dataset_text, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, inputs):
        self.input_ids = inputs['input_ids']

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'labels': self.input_ids[idx]
        }

dataset = CustomDataset(inputs)

In [ ]:
dataset.input_ids[0]

tensor([   16,  1058,   220,   166,   116,   108,   168,   243,   230,  9508,
        23821,   243,   234,   168,   243,   226,    30,   198,    17,  1058,
          220,   166,   115,   116,   167,   253,   120,   220,   166,   116,
          108,   168,   243,   230, 31619,   103,   101,   167,    98,   112,
          167,   232,   242, 23821,  8955,   167,   252,   234,   167,   237,
          226, 23821,   252,   230,   168,   244,   112,    30,   198,    16,
         1058,   220,   166,   115,   116,   167,   252,   246,   167,   237,
          226, 23821,   252,   230,   166,   110,   254,   168,   100,   222,
           30,   198,    17,  1058, 23821,   248,   242,   168,    99,   246,
        23821,   246,   230,   167,   232,    98,   168,   245,   238,   167,
          237,   226, 23821,   245,   226,   168,   110,   255, 31619,   100,
          236, 35975,   112, 31619,   224,   246,   168,   246,    97,   168,
          252,   244,   168,   243,   226,   198,    16,  1058, 

In [ ]:
# 모델 설정
model = GPT2LMHeadModel.from_pretrained('gpt2')

# 데이터 콜레이터 설정 #특히, 배치 내에서 시퀀스의 길이를 맞추고, 모델이 입력으로 사용할 수 있는 형태로 데이터를 준비하는 데 중요한 역할
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
print(data_collator)

DataCollatorForLanguageModeling(tokenizer=GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}, mlm=False, mlm_probability=0.15, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')


In [ ]:


# 훈련 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    warmup_steps=100,
    weight_decay=0.01,
    prediction_loss_only=True
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

# 모델 학습
trainer.train()

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Step,Training Loss


TrainOutput(global_step=3, training_loss=2.472482363382975, metrics={'train_runtime': 13.7196, 'train_samples_per_second': 0.219, 'train_steps_per_second': 0.219, 'total_flos': 783876096000.0, 'train_loss': 2.472482363382975, 'epoch': 3.0})

In [ ]:
# GPU 또는 CPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 챗봇 응답 생성 함수
def generate_response(model, tokenizer, input_text, max_length=50):
    model.eval()
    inputs = tokenizer.encode(input_text, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response



In [ ]:
tokenizer.eos_token_id

50256

In [ ]:
# 예시 입력 텍스트
input_text = "머먹었어?"

# 모델을 사용하여 응답 생성
response = generate_response(model, tokenizer, input_text)
print(f"Response: {response}")

Response: 머먹었어?

아리리었어?

아리리었어?



In [ ]:
# 모델을 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
# 모델 설정
model = GPT2LMHeadModel.from_pretrained('gpt2')

# 모델을 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 데이터 콜레이터 설정
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 훈련 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=100,
    per_device_train_batch_size=30,
    per_device_eval_batch_size=30,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    warmup_steps=100,
    weight_decay=0.01,
    prediction_loss_only=True,
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

# 모델 학습
trainer.train()

# 챗봇 응답 생성 함수
def generate_response(model, tokenizer, input_text, max_length=50):
    model.eval()
    inputs = tokenizer.encode(input_text, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# 예시 입력 텍스트
input_text = "어떤 영화를 추천해줄래?"

# 모델을 사용하여 응답 생성
response = generate_response(model, tokenizer, input_text)
print(f"Response: {response}")

Step,Training Loss
10,2.591900
20,2.364700
30,2.080700
40,1.754100
50,1.406700
60,1.003400
70,0.642700
80,0.364100
90,0.197300
100,0.122600


Response: 어떤 영화를 추천해줄래?
어떤 잭! 아니지�


In [ ]:
len(json_files)

4276

In [ ]:
from transformers import GPT2Tokenizer, DataCollatorForLanguageModeling
import torch

# 토크나이저 로드
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# 데이터 수집기 정의
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 예제 데이터
texts = ["Hello, how are you?", "I am fine, thank you!", "What about you?"]
tokenizer.pad_token = tokenizer.eos_token
# 텍스트를 토큰화
inputs = tokenizer(texts, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

# inputs가 dictionary 형식으로 되어 있으므로 list of dict 형식으로 변환합니다.
inputs_list = [{'input_ids': input_id, 'attention_mask': att_mask} for input_id, att_mask in zip(inputs['input_ids'], inputs['attention_mask'])]

# Data Collator 사용하여 배치 데이터 준비
batch = data_collator(inputs_list)

# 결과 출력
print(batch)

{'input_ids': tensor([[15496,    11,   703,  ..., 50256, 50256, 50256],
        [   40,   716,  3734,  ..., 50256, 50256, 50256],
        [ 2061,   546,   345,  ..., 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[15496,    11,   703,  ...,  -100,  -100,  -100],
        [   40,   716,  3734,  ...,  -100,  -100,  -100],
        [ 2061,   546,   345,  ...,  -100,  -100,  -100]])}


In [ ]:
len(all_texts)

4276

In [ ]:
!pip install torch transformers

In [ ]:
import os
import json
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from google.colab import drive

# 구글 드라이브 마운트
drive.mount('/content/drive')

# 데이터 경로 설정 (구글 드라이브 내의 경로)
data_path = '/content/drive/MyDrive/movie dialogue'

# 모든 JSON 파일의 경로를 리스트로 저장
json_files = [os.path.join(data_path, f) for f in os.listdir(data_path) if f.endswith('.json')]

# 데이터를 저장할 리스트
dialogue_pairs = []

# 각 JSON 파일을 불러와서 데이터 추가
for file_path in json_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        for item in data['info']:
            text = item['annotations']['text']
            # 대화 쌍 추출 (여기서는 간단히 첫 번째 줄과 두 번째 줄을 대화 쌍으로 가정)
            lines = text.split('\n')
            if len(lines) >= 2:
                dialogue_pairs.append((lines[0], lines[1]))

print(f'Loaded {len(dialogue_pairs)} dialogue pairs.')

# 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained('t5-small')

# 대화 데이터를 추출하여 토큰화
input_texts = [pair[0] for pair in dialogue_pairs]
target_texts = [pair[1] for pair in dialogue_pairs]

# 토큰화된 데이터를 저장할 리스트
inputs = tokenizer(input_texts, max_length=512, truncation=True, padding='max_length', return_tensors='pt')
targets = tokenizer(target_texts, max_length=512, truncation=True, padding='max_length', return_tensors='pt')

# 데이터셋 생성
class DialogueDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs['input_ids'])

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs['input_ids'][idx],
            'attention_mask': self.inputs['attention_mask'][idx],
            'labels': self.targets['input_ids'][idx]
        }

dataset = DialogueDataset(inputs, targets)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded 4276 dialogue pairs.


In [ ]:
# 모델 설정
model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')

# 훈련 전 캐시 초기화
torch.cuda.empty_cache()

# 훈련 설정
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,  # 배치 크기를 줄임
    per_device_eval_batch_size=8,   # 배치 크기를 줄임
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    gradient_accumulation_steps=4,  # Gradient Accumulation 사용
    fp16=True  # Mixed Precision Training 사용
)

# Trainer 설정
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# 모델 학습
trainer.train()

# 훈련 후 캐시 초기화
torch.cuda.empty_cache()

Step,Training Loss
10,15.419900
20,15.656500
30,15.454400
40,15.225600
50,13.983400
60,13.101200
70,12.100000
80,10.330800
90,8.197200
100,6.347500


In [ ]:
def generate_response(model, tokenizer, input_text):
    # 모델이 사용 중인 장치 가져오기
    device = next(model.parameters()).device

    # 입력 텍스트를 토큰화하고 텐서를 모델이 위치한 장치로 이동
    inputs = tokenizer(input_text, return_tensors='pt', truncation=True, padding='max_length', max_length=512).to(device)

    # 응답 생성
    with torch.no_grad():
        outputs = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=50)

    # 출력 텐서를 디코딩하여 응답 생성
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# 예시 입력 텍스트
input_text = "어떤 영화를 추천해줄래?"

# 모델을 사용하여 응답 생성
response = generate_response(model, tokenizer, input_text)
print(f"Response: {response}")

Response:     


In [ ]:
#추천 하는건 머신러닝이고
# 대화를 하는 건 ...